### 1. Settings

In [1]:
#####################################
##########  DEPENDECIES ############
#####################################

import os
import pickle
import sys
sys.path.append("../")

from tqdm import tqdm # type: ignore
from datetime import date

import evaluate

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from utils.sampling import create_splits, prep_cv_validation
from utils.training import cv_cluster_set, cv_training_epochs_sets, test_cluster_set
from utils.training import results_dict_todf, cv_step_2, full_step_2, test_training_epochs_sets
from utils.inference import meta_predict, create_ensemble_map, ensemble_compute

tqdm.pandas()
import warnings
warnings.filterwarnings("ignore")

os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB_DISABLED"] = "true"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch

#####################################
############  CONSTANTS #############
#####################################

RS = 42
BATCH_SIZE = 16
DECODER_LENGTH = 30
ENCODER_LENGTH = 30
MODEL_NAME = "Salesforce/codet5-base-multi-sum"

FULL_TRAIN_ARGS = {
    "BATCH_SIZE": BATCH_SIZE,
    "DECODER_LENGTH": DECODER_LENGTH,
    "ENCODER_LENGTH": ENCODER_LENGTH,
    "SEQ_TRAINER_ARGS": {
        "overwrite_output_dir": True,
        "num_train_epochs": [0, 1, 2, 5, 10],
        "do_train": True,
        "do_eval": True,
        "per_device_train_batch_size": 4,
        "per_device_eval_batch_size": 4,
        "learning_rate": 1e-5,
        "warmup_steps": 500,
        "weight_decay": 0.1,
        "label_smoothing_factor": 0.1,
        "predict_with_generate": True,
        "logging_steps": 100,
        "save_total_limit": 1,
        "save_strategy": "no",
        "logging_strategy": "epoch",
        "evaluation_strategy": "epoch",
        "load_best_model_at_end": False,
        "output_dir" : 'reports/results',
        "logging_dir" : "reports/logs",
    },
}

experiment_config = {
    "DATA_STR" : "20240908",
    "RS" : RS,
    "DRIFT_TYPE" : "no_drift",
    "NFOLD" : 3,
    "FULL_TRAIN_ARGS" : FULL_TRAIN_ARGS,
    "MODEL_NAME" : MODEL_NAME,
    "CLUSTER_EPOCHS" : 3,
    "CLUSTER_SET_ID" : [0, 3, [0, 3,]],
    "TRAIN_SIZE" : 7000,
    "TEST_SIZE" : 2500,
}
experiment_config["ANALYSIS_POSTFIX"] = f"mined_{experiment_config['DRIFT_TYPE']}_{str(date.today())}"
experiment_config["FULL_TRAIN_ARGS"]["SEQ_TRAINER_ARGS"]["output_dir"] += "/" + experiment_config["ANALYSIS_POSTFIX"] 
experiment_config["FULL_TRAIN_ARGS"]["SEQ_TRAINER_ARGS"]["logging_dir"] += "/" + experiment_config["ANALYSIS_POSTFIX"] 

if not os.path.exists(experiment_config["FULL_TRAIN_ARGS"]["SEQ_TRAINER_ARGS"]["logging_dir"]):
    os.mkdir(experiment_config["FULL_TRAIN_ARGS"]["SEQ_TRAINER_ARGS"]["logging_dir"])

if not os.path.exists(experiment_config["FULL_TRAIN_ARGS"]["SEQ_TRAINER_ARGS"]["output_dir"]):
    os.mkdir(experiment_config["FULL_TRAIN_ARGS"]["SEQ_TRAINER_ARGS"]["output_dir"])

tokenizer = AutoTokenizer.from_pretrained(experiment_config["MODEL_NAME"], skip_special_tokens=False)
model = AutoModelForSeq2SeqLM.from_pretrained(experiment_config["MODEL_NAME"])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
rouge = evaluate.load('rouge')

experiment_config["ANALYSIS_POSTFIX"] = f"mined_{experiment_config['DRIFT_TYPE']}_2024-09-09"
RUN_BASE_TRAINING = False
t_models=["catboost"]

[nltk_data] Downloading package punkt to
[nltk_data]     /home/RDC/zinovyee.hub/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 2. Conala data. Preprocessing. 

In [2]:
sampling_dict = create_splits(experiment_config=experiment_config, tokenizer=tokenizer, train_size=experiment_config["TRAIN_SIZE"], test_size=experiment_config["TEST_SIZE"], cluster_id=4)
train_dataset, test_data, test_df, train_df = sampling_dict["train_data"], sampling_dict["test_data"], sampling_dict["test_df"], sampling_dict["train_df"]

splits, questions_list = prep_cv_validation(train_dataset=train_dataset, 
                            experiment_config=experiment_config)

Train Data:  (7000, 11)
Test Data:  (2500, 11)
Train Data: Cluster cluster
1    3280
0    1352
2    1325
3    1043
Name: count, dtype: int64
Test Data: Cluster cluster
1    1156
2     490
0     441
3     413
Name: count, dtype: int64


Filter:   0%|          | 0/7000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/7000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Fold 0
Fold 1
Fold 2


In [3]:
if RUN_BASE_TRAINING:
    fold_results = cv_training_epochs_sets(experiment_config=experiment_config,
                                splits=splits,
                                questions_list=questions_list,
                                train_dataset=train_dataset,
                                tokenizer=tokenizer)

    with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/cv_fold_epoch_set.pickle', 'wb') as handle:
        pickle.dump(fold_results, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [4]:
if RUN_BASE_TRAINING:
    with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/cv_fold_epoch_set.pickle', 'rb') as handle:
        fold_results = pickle.load(handle)
    
    for cluster_idx in experiment_config["CLUSTER_SET_ID"]:
        fold_results = cv_cluster_set(experiment_config=experiment_config,
                                                splits=splits,
                                                questions_list=questions_list,
                                                train_dataset=train_dataset,
                                                tokenizer=tokenizer,
                                                fold_results=fold_results,
                                                cluster_id=cluster_idx)

    cv_df = results_dict_todf(fold_results)

    ########## SAVE THE FILE

    with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/cv_step1.pickle', 'wb') as handle:
        pickle.dump(cv_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [5]:
if RUN_BASE_TRAINING:
    print("Mean")
    print(cv_df.groupby(["model_set"])["rouge"].mean())

    print("STD")
    print(cv_df.groupby("model_set")["rouge"].std())

### Step 2. Learn performance

In [6]:
########## LOAD CV RESULTS

import pickle
with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/cv_step1.pickle', 'rb') as handle:
    cv_df = pickle.load(handle)

########## RUN STEP 2 ON CV

cv_df, model_results = cv_step_2(experiment_config=experiment_config, cv_df=cv_df, t_models=t_models)

with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/s2_model_results.pickle', 'wb') as handle:
    pickle.dump(model_results, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/cv_results.pickle', 'wb') as handle:
    pickle.dump(cv_df, handle, protocol=pickle.HIGHEST_PROTOCOL)


0
catboost
Learning rate set to 0.072537
0:	learn: 0.1639212	total: 65.8ms	remaining: 1m 5s
1:	learn: 0.1629041	total: 72.7ms	remaining: 36.3s
2:	learn: 0.1620000	total: 79.7ms	remaining: 26.5s
3:	learn: 0.1611945	total: 86.7ms	remaining: 21.6s
4:	learn: 0.1605028	total: 93.2ms	remaining: 18.5s
5:	learn: 0.1598611	total: 100ms	remaining: 16.6s
6:	learn: 0.1593115	total: 107ms	remaining: 15.1s
7:	learn: 0.1588391	total: 114ms	remaining: 14.1s
8:	learn: 0.1584001	total: 122ms	remaining: 13.4s
9:	learn: 0.1580357	total: 129ms	remaining: 12.7s
10:	learn: 0.1577346	total: 136ms	remaining: 12.2s
11:	learn: 0.1574196	total: 143ms	remaining: 11.8s
12:	learn: 0.1571376	total: 150ms	remaining: 11.4s
13:	learn: 0.1569133	total: 157ms	remaining: 11s
14:	learn: 0.1567282	total: 164ms	remaining: 10.8s
15:	learn: 0.1565316	total: 171ms	remaining: 10.5s
16:	learn: 0.1563601	total: 177ms	remaining: 10.3s
17:	learn: 0.1562147	total: 184ms	remaining: 10.1s
18:	learn: 0.1560932	total: 192ms	remaining: 9.9

In [7]:
### TO SAVE THE VECTORIZER AND STEP 2 MODELS

with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/cv_results.pickle', 'rb') as handle:
    cv_df = pickle.load(handle)


print("Mean")
print(cv_df.groupby(["model_set"])["catboost_perf_hat"].mean())

print("STD")
print(cv_df.groupby("model_set")["catboost_perf_hat"].std())


full_step_2(cv_df=cv_df, 
            experiment_config=experiment_config, t_models=t_models)

Mean
model_set
0                 0.132312
1                 0.273133
2                 0.274084
5                 0.279276
10                0.279877
cluster_[0, 3]    0.273351
cluster_[0]       0.200182
cluster_[3]       0.253990
ensemble          0.280119
Name: catboost_perf_hat, dtype: float64
STD
model_set
0                 0.046651
1                 0.047404
2                 0.047423
5                 0.047317
10                0.047236
cluster_[0, 3]    0.047359
cluster_[0]       0.046675
cluster_[3]       0.046849
ensemble          0.047344
Name: catboost_perf_hat, dtype: float64
catboost
Learning rate set to 0.077338
0:	learn: 0.1648190	total: 10.4ms	remaining: 10.3s
1:	learn: 0.1637201	total: 19.1ms	remaining: 9.55s
2:	learn: 0.1627315	total: 27.5ms	remaining: 9.14s
3:	learn: 0.1619450	total: 36.1ms	remaining: 8.99s
4:	learn: 0.1612320	total: 44.4ms	remaining: 8.84s
5:	learn: 0.1605757	total: 52.5ms	remaining: 8.7s
6:	learn: 0.1600621	total: 61ms	remaining: 8.65s
7:	learn: 0.

# TEST

In [8]:
# sampling_dict = create_splits(experiment_config=experiment_config, tokenizer=tokenizer, test=True, train_size=100, test_size=25, cluster_id=4)
# train_dataset, test_data, test_df, train_df = sampling_dict["train_data"], sampling_dict["test_data"], sampling_dict["test_df"], sampling_dict["train_df"]

# splits, questions_list = prep_cv_validation(train_dataset=train_dataset, 
#                             experiment_config=experiment_config)

In [9]:
with open(f"reports/results/{experiment_config['ANALYSIS_POSTFIX']}/cv_results.pickle", "rb") as handle:
    cv_resutls = pickle.load(handle)

base_models_list = list(cv_resutls.model_set.unique())
base_models_list.pop(-1)

'ensemble'

In [10]:
meta_preds_df = meta_predict(experiment_config=experiment_config, 
                            test_df=test_df,
                            base_models_names=base_models_list,
                            t_models=t_models)

########## SAVE THE FILE

with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/test_step2.pickle', 'wb') as handle:
    pickle.dump(meta_preds_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

catboost


In [11]:
with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/test_step2.pickle', 'rb') as handle:
    meta_preds_df = pickle.load(handle)
    
meta_preds_df.groupby("model_set").catboost_preds.mean()

model_set
0                 0.134053
1                 0.274264
2                 0.276397
5                 0.281247
10                0.281165
cluster_[0, 3]    0.274264
cluster_[0]       0.202445
cluster_[3]       0.255489
Name: catboost_preds, dtype: float64

In [19]:
meta_preds_df.groupby("model_set").catboost_preds.std()

model_set
0                 0.044460
1                 0.044276
2                 0.044232
5                 0.044225
10                0.044235
cluster_[0, 3]    0.044276
cluster_[0]       0.043959
cluster_[3]       0.043859
Name: catboost_preds, dtype: float64

In [12]:
optimal_ensemble_map, ensemble_val_estim = create_ensemble_map(meta_preds_df=meta_preds_df, 
                                                                t_model_name=t_models[0])

In [22]:
ensemble_val_estim.catboost_preds.mean()

0.28141443659554277

In [23]:
ensemble_val_estim.catboost_preds.std()

0.044273420296207566

In [13]:
if RUN_BASE_TRAINING:
    test_result_df = test_training_epochs_sets(experiment_config=experiment_config,
                                test_df=test_df,
                                test_data=test_data,
                                train_data=train_dataset,
                                tokenizer=tokenizer)

    with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/test_epoch_set.pickle', 'wb') as handle:
        pickle.dump(test_result_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [14]:
if RUN_BASE_TRAINING:
    with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/test_epoch_set.pickle', 'rb') as handle:
        test_result_df = pickle.load(handle)

    test_result_df = test_result_df.rename(columns={"epoch_set": "model_set"})

    for cluster_idx in experiment_config["CLUSTER_SET_ID"]:
        test_result_df = test_cluster_set(experiment_config=experiment_config,
                                        test_df=test_df,
                                        test_data=test_data,
                                        tokenizer=tokenizer,
                                        results_df=test_result_df,
                                        cluster_id=cluster_idx,
                                        train_df=train_df)

    ########## SAVE THE FILE

    with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/test_results.pickle', 'wb') as handle:
        pickle.dump(test_result_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [17]:
with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/test_results.pickle', 'rb') as handle:
        test_result_df = pickle.load(handle)

### ENSEMBLE COMPUTE
test_result_df = ensemble_compute(test_result_df=test_result_df,
                                  optimal_ensemble_map=optimal_ensemble_map)

########## ROUGE PER SETTING

print("Mean")
print(test_result_df.groupby("model_set")["rouge"].mean())

print("STD")
print(test_result_df.groupby("model_set")["rouge"].std())

print(test_result_df.loc[test_result_df["model_set"] ==test_result_df["opt_es_id"]].opt_es_id.value_counts())

with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/test_results_full.pickle', 'wb') as handle:
    pickle.dump(test_result_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

Mean
model_set
0                 0.123299
1                 0.276431
2                 0.282285
5                 0.286905
10                0.284072
cluster_[0, 3]    0.266360
cluster_[0]       0.202202
cluster_[3]       0.251622
ensemble          0.286670
Name: rouge, dtype: float64
STD
model_set
0                 0.133633
1                 0.162703
2                 0.162382
5                 0.165110
10                0.164032
cluster_[0, 3]    0.157822
cluster_[0]       0.158643
cluster_[3]       0.150236
ensemble          0.165231
Name: rouge, dtype: float64
opt_es_id
5     20889
10     1602
2         9
Name: count, dtype: int64
